# Notebook for testing

### scrape ird documents

Using scrapy to scrape ird documents.

In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess

class IrdTableSpider(scrapy.Spider):
    name = "ird_table_spider"
    start_urls = ['https://www.ird.gov.hk/eng/ppr/arc.htm']

    def parse(self, response):
        # Select the table rows, skip the header
        rows = response.xpath('//table[contains(@class, "border_table")]/tbody/tr[position()>1]')
        for row in rows:
            case_no = row.xpath('td[1]/a/text()').get()
            case_link = row.xpath('td[1]/a/@href').get()
            provision = row.xpath('td[2]/text()').get()
            # Get all <li> items in the 3rd column
            index_items = row.xpath('td[3]//li/text()').getall()
            yield {
                'case_no': case_no,
                'case_link': response.urljoin(case_link) if case_link else None,
                'provision': provision.strip() if provision else None,
                'index': [item.strip() for item in index_items],
            }

def run_spider():
    process = CrawlerProcess(settings={
        "LOG_LEVEL": "ERROR",
        "FEEDS": {"results.json": {"format": "json"}},
    })
    process.crawl(IrdTableSpider)
    process.start()

if __name__ == "__main__":
    run_spider()

In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess

class IrdCaseContentSpider(scrapy.Spider):
    name = "ird_case_content_spider"
    start_urls = [f'https://www.ird.gov.hk/eng/ppr/advance{i}.htm' for i in [13, 16, 26, 44]]

    def parse(self, response):
        # get the title
        title = response.css('div.content-title-div h1.content-title::text').get()
        print(f'title: {title}')

        yield {
            'title': title.strip() if title else None,
        }

def run_spider():
    process = CrawlerProcess(settings={
        "LOG_LEVEL": "ERROR",
        "FEEDS": {"results.json": {"format": "json"}},
    })
    process.crawl(IrdCaseContentSpider)
    process.start()

if __name__ == "__main__":
    run_spider()

RuntimeError: This event loop is already running

: 

Using Selenium to scrape ird documents.

In [2]:
# write a python script using selenium to scrape the table data from https://www.ird.gov.hk/eng/ppr/arc.htm and save it as a json file
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import json
import time

In [ ]:
def scrape_ird_table():
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get('https://www.ird.gov.hk/eng/ppr/arc.htm')
    time.sleep(5)  # wait for the page to load

    table = driver.find_element(By.CSS_SELECTOR, 'table.border_table')
    rows = table.find_elements(By.TAG_NAME, 'tr')[1:]  # skip header row

    results = []
    for row in rows:
        cols = row.find_elements(By.TAG_NAME, 'td')

        case_no = cols[0].find_element(By.TAG_NAME, 'a').get_attribute('innerHTML') if cols[0].find_elements(By.TAG_NAME, 'a') else None
        case_link = cols[0].find_element(By.TAG_NAME, 'a').get_attribute('href') if cols[0].find_elements(By.TAG_NAME, 'a') else None
        provision = cols[1].get_attribute('innerHTML') if cols[1] else None
        index_items = [li.get_attribute('innerHTML') for li in cols[2].find_elements(By.TAG_NAME, 'li')]
        results.append({
            'case_no': case_no,
            'case_link': case_link,
            'provision': provision,
            'index': index_items,
        })

    with open('./output_files/ird_results.json', 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

    driver.quit()

if __name__ == "__main__":
    scrape_ird_table()

<selenium.webdriver.remote.webelement.WebElement (session="d6d308dab4a380b0608ce8dbce7a0b47", element="f.E406D1A056F22A83968F0DBC1D753D7B.d.D010D05BAB9ABE93990C532DA6EDC61D.e.70")>
<selenium.webdriver.remote.webelement.WebElement (session="d6d308dab4a380b0608ce8dbce7a0b47", element="f.E406D1A056F22A83968F0DBC1D753D7B.d.D010D05BAB9ABE93990C532DA6EDC61D.e.68")>
[<selenium.webdriver.remote.webelement.WebElement (session="d6d308dab4a380b0608ce8dbce7a0b47", element="f.E406D1A056F22A83968F0DBC1D753D7B.d.D010D05BAB9ABE93990C532DA6EDC61D.e.71")>, <selenium.webdriver.remote.webelement.WebElement (session="d6d308dab4a380b0608ce8dbce7a0b47", element="f.E406D1A056F22A83968F0DBC1D753D7B.d.D010D05BAB9ABE93990C532DA6EDC61D.e.72")>]
<selenium.webdriver.remote.webelement.WebElement (session="d6d308dab4a380b0608ce8dbce7a0b47", element="f.E406D1A056F22A83968F0DBC1D753D7B.d.D010D05BAB9ABE93990C532DA6EDC61D.e.76")>
<selenium.webdriver.remote.webelement.WebElement (session="d6d308dab4a380b0608ce8dbce7a0b47"

### Download ird pdf documents

In [16]:
import subprocess

In [18]:
i = 63
f"{i:02d}"

'63'

In [ ]:
# download pdf documents to output_files/ird_pdfs

destination_directory = "./output_files/ird_pdfs"

try:
    for i in range(1, 64):
        pdf_url = f"https://www.ird.gov.hk/eng/pdf/dipn{i:02d}.pdf"
        wget_command = ["wget", "-P", destination_directory, pdf_url]
        subprocess.run(wget_command, check=True, capture_output=True, text=True)
        print(f"File downloaded successfully to: {destination_directory}")
except subprocess.CalledProcessError as e:
    print(f"Error downloading file: {e}")
    print(f"Stderr: {e.stderr}")

File downloaded successfully to: ./output_files/ird_pdfs
File downloaded successfully to: ./output_files/ird_pdfs
File downloaded successfully to: ./output_files/ird_pdfs
File downloaded successfully to: ./output_files/ird_pdfs
File downloaded successfully to: ./output_files/ird_pdfs
File downloaded successfully to: ./output_files/ird_pdfs
File downloaded successfully to: ./output_files/ird_pdfs
File downloaded successfully to: ./output_files/ird_pdfs
File downloaded successfully to: ./output_files/ird_pdfs
File downloaded successfully to: ./output_files/ird_pdfs
File downloaded successfully to: ./output_files/ird_pdfs
File downloaded successfully to: ./output_files/ird_pdfs
File downloaded successfully to: ./output_files/ird_pdfs
File downloaded successfully to: ./output_files/ird_pdfs
File downloaded successfully to: ./output_files/ird_pdfs
File downloaded successfully to: ./output_files/ird_pdfs
File downloaded successfully to: ./output_files/ird_pdfs
File downloaded successfully to

In [20]:
try:
    # download pdf 13A document
    pdf_url = f"https://www.ird.gov.hk/eng/pdf/dipn13a.pdf"
    wget_command = ["wget", "-P", destination_directory, pdf_url]
    subprocess.run(wget_command, check=True, capture_output=True, text=True)
    print(f"File downloaded successfully to: {destination_directory}")
except subprocess.CalledProcessError as e:
    print(f"Error downloading file: {e}")
    print(f"Stderr: {e.stderr}")

File downloaded successfully to: ./output_files/ird_pdfs


### Scrape pdf files metadata

In [2]:
import re

In [6]:
# utils functions
def extract_only_alphanumeric(text: str) -> str:
    '''
    Extract only the first continuous sequence of digits and alphabets from the input text.

    Args:
        text (str): The input text from which to extract numbers.
    Returns:
        str: The extracted number as a string, or an empty string if no digits and alphabets are found.
    '''

    match = re.search(r'[A-Za-z0-9]+', text)
    return match.group(0) if match else ''
    

def remove_html_tags(text: str) -> str:
    '''
    Remove HTML tags from the input text.

    Args:
        text (str): The input text containing HTML tags.
    Returns:
        str: The text with HTML tags removed.
    '''
    
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)
    

In [7]:
def scrape_ird_pdf_metadata():
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get('https://www.ird.gov.hk/eng/ppr/dip.htm')
    time.sleep(5)  # wait for the page to load

    table = driver.find_element(By.CSS_SELECTOR, 'table.border_table')
    rows = table.find_elements(By.TAG_NAME, 'tr')[1:]  # skip header row

    results = []
    for row in rows:
        cols = row.find_elements(By.TAG_NAME, 'td')

        pdf_no = cols[0].find_element(By.TAG_NAME, 'a').get_attribute('innerHTML') if cols[0].find_elements(By.TAG_NAME, 'a') else None
        pdf_link = cols[0].find_element(By.TAG_NAME, 'a').get_attribute('href') if cols[0].find_elements(By.TAG_NAME, 'a') else None
        pdf_notes = cols[1].find_element(By.TAG_NAME, 'a').get_attribute('innerHTML') if cols[1] else None
        pdf_notes = remove_html_tags(text=pdf_notes) if pdf_notes else None
        pdf_date = cols[2].get_attribute('innerHTML') if cols[2] else None
        results.append({
            'pdf_no': extract_only_alphanumeric(text=pdf_no),
            'pdf_link': pdf_link,
            'pdf_notes': pdf_notes,
            'index': pdf_date,
        })

    with open('./output_files/ird_pdf_results.json', 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

    driver.quit()

if __name__ == "__main__":
    scrape_ird_pdf_metadata()

### Parsing pdf file using llama-parse

In [8]:
import nest_asyncio
nest_asyncio.apply()
import os
from dotenv import load_dotenv
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
load_dotenv(override=True)

# create llama parse instance
parser = LlamaParse(
    api_key=os.getenv('LLAMAINDEX_KEY'),
    result_type="markdown",
    num_workers=4,
    language='en',
    verbose=True
)

# start to parse the pdf files in output_files/ird_pdfs
# using SimpleDirectoryReader
file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(
    input_dir="./output_files/ird_pdfs",
    file_extractor=file_extractor
).load_data()

documents


2025-08-23 21:54:29,889 - INFO - HTTP Request: POST https://api.cloud.llamaindex.ai/api/parsing/upload "HTTP/1.1 200 OK"


Started parsing the file under job_id 62dca517-8c4a-435e-bc2c-25820356aedf


2025-08-23 21:54:31,239 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/62dca517-8c4a-435e-bc2c-25820356aedf "HTTP/1.1 200 OK"
2025-08-23 21:54:33,620 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/62dca517-8c4a-435e-bc2c-25820356aedf "HTTP/1.1 200 OK"
2025-08-23 21:54:36,967 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/62dca517-8c4a-435e-bc2c-25820356aedf "HTTP/1.1 200 OK"
2025-08-23 21:54:41,310 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/62dca517-8c4a-435e-bc2c-25820356aedf "HTTP/1.1 200 OK"
2025-08-23 21:54:48,571 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/62dca517-8c4a-435e-bc2c-25820356aedf "HTTP/1.1 200 OK"
2025-08-23 21:54:54,422 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/62dca517-8c4a-435e-bc2c-25820356aedf "HTTP/1.1 200 OK"
2025-08-23 21:54:55,088 - INFO - HTTP Request: GET https://api.cloud.llamain

Error while parsing the file '<bytes/buffer>': Event loop is closed


2025-08-23 21:55:00,472 - INFO - HTTP Request: POST https://api.cloud.llamaindex.ai/api/parsing/upload "HTTP/1.1 200 OK"


...

2025-08-23 22:09:56,911 - INFO - HTTP Request: POST https://api.cloud.llamaindex.ai/api/parsing/upload "HTTP/1.1 200 OK"


Started parsing the file under job_id cad1f0ba-f81a-4803-8770-321a9af9ac1d


2025-08-23 22:09:58,291 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cad1f0ba-f81a-4803-8770-321a9af9ac1d "HTTP/1.1 200 OK"
2025-08-23 22:10:00,691 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cad1f0ba-f81a-4803-8770-321a9af9ac1d "HTTP/1.1 200 OK"
2025-08-23 22:10:04,057 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cad1f0ba-f81a-4803-8770-321a9af9ac1d "HTTP/1.1 200 OK"
2025-08-23 22:10:08,443 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cad1f0ba-f81a-4803-8770-321a9af9ac1d "HTTP/1.1 200 OK"
2025-08-23 22:10:15,554 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cad1f0ba-f81a-4803-8770-321a9af9ac1d "HTTP/1.1 200 OK"
2025-08-23 22:10:21,434 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cad1f0ba-f81a-4803-8770-321a9af9ac1d "HTTP/1.1 200 OK"
2025-08-23 22:10:30,739 - INFO - HTTP Request: GET https://api.cloud.llamain

Error while parsing the file '<bytes/buffer>': Event loop is closed


[Document(id_='57e57e01-b5b6-4b9e-8e37-8a97d6b68470', embedding=None, metadata={'file_path': '/Users/tongcc/dev/projects/interview/ird_data_pipeline/tests/output_files/ird_pdfs/dipn01.pdf', 'file_name': 'dipn01.pdf', 'file_type': 'application/pdf', 'file_size': 502041, 'creation_date': '2020-09-30', 'last_modified_date': '2020-09-30'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='\n# Inland Revenue Department\n\n# The Government of the Hong Kong Special Administrative Region of the People’s Republic of China\n\n# DEPARTMENTAL INTERPRETATION AND PRACTICE NOTES\n\n# NO. 1 (REVISED)\n\n# PROFITS TAX\n\n# PART A: COMPUTING ASSESSABLE PROFIT

In [1]:
import pickle

In [10]:
# pickle dump the documents
with open('./objects/ird_llamaindex_pdf_documents.pkl', 'wb') as f:
    pickle.dump(documents, f)

In [2]:
# pickle load the documents
with open('./objects/ird_llamaindex_pdf_documents.pkl', 'rb') as f:
    documents_loaded = pickle.load(f)

documents_loaded[:5]


[Document(id_='57e57e01-b5b6-4b9e-8e37-8a97d6b68470', embedding=None, metadata={'file_path': '/Users/tongcc/dev/projects/interview/ird_data_pipeline/tests/output_files/ird_pdfs/dipn01.pdf', 'file_name': 'dipn01.pdf', 'file_type': 'application/pdf', 'file_size': 502041, 'creation_date': '2020-09-30', 'last_modified_date': '2020-09-30'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='\n# Inland Revenue Department\n\n# The Government of the Hong Kong Special Administrative Region of the People’s Republic of China\n\n# DEPARTMENTAL INTERPRETATION AND PRACTICE NOTES\n\n# NO. 1 (REVISED)\n\n# PROFITS TAX\n\n# PART A: COMPUTING ASSESSABLE PROFIT

In [3]:
documents = documents_loaded

In [4]:
len(documents)

1142

### Converting pdf files to markdown files

##### using pandoc

In [14]:
import subprocess

In [18]:
try:
    # using subprocess with pandoc to convert pdf to markdown in tests/output_files/ird_pdfs and output the markdown files in ird_md_pandoc
    directory_path = "./output_files/ird_pdfs"
    output_directory = "./output_files/ird_md_pandoc"
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(directory_path, filename)
            md_filename = filename.replace(".pdf", ".md")
            md_path = os.path.join(output_directory, md_filename)
            print(f'pdf_path: {pdf_path}, md_path: {md_path}')
            pandoc_command = ["Pandoc", "-f", "pdf", "-t", "markdown", "-o", md_path, pdf_path]
            subprocess.run(pandoc_command, check=True, capture_output=True, text=True)
            print(f"Converted {pdf_path} to {md_path}")
except subprocess.CalledProcessError as e:
    print(f"Error downloading file: {e}")
    print(f"Stderr: {e.stderr}")

pdf_path: ./output_files/ird_pdfs/dipn19.pdf, md_path: ./output_files/ird_md_pandoc/dipn19.md
Error downloading file: Command '['Pandoc', '-f', 'pdf', '-t', 'markdown', '-o', './output_files/ird_md_pandoc/dipn19.md', './output_files/ird_pdfs/dipn19.pdf']' returned non-zero exit status 21.
Stderr: Unknown input format pdf
Pandoc can convert to PDF, but not from PDF.



##### using markitdown

In [25]:
from markitdown import MarkItDown

# Create a converter instance
converter = MarkItDown()

# Convert PDF to Markdown
directory_path = "./output_files/ird_pdfs"
output_directory = "./output_files/ird_md_markitdown"
for filename in os.listdir(directory_path):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(directory_path, filename)
        md_filename = filename.replace(".pdf", ".md")
        md_path = os.path.join(output_directory, md_filename)
        markdown_text = converter.convert(pdf_path)

        # Persist the result
        with open(md_path, "w", encoding="utf-8") as f:
            f.write(markdown_text.text_content)


In [24]:
markdown_text.text_content

"Inland Revenue Department\nThe Government of the Hong Kong Special Administrative Region\nof the People's Republic of China\n\nDEPARTMENTAL INTERPRETATION AND PRACTICE NOTES\n\nNO. 14 (REVISED)\n\nPROPERTY TAX\n\nThese  notes  are  issued  for  the  information  of  taxpayers  and  their  tax\nrepresentatives.  They  contain  the  Department’s  interpretation  and  practices  in\nrelation to the law as it stood at the date of publication.  Taxpayers are reminded\nthat their right of objection against the assessment and their right of appeal to the\nCommissioner,  the  Board  of  Review  or  the  Court  are  not  affected  by  the\napplication of these notes.\n\nThese notes replace those issued in February 2005.\n\nCHU Yam-yuen\nCommissioner of Inland Revenue\n\nMarch 2011\n\nOur web site : www.ird.gov.hk\n\n\x0cDEPARTMENTAL INTERPRETATION AND PRACTICE NOTES\n\nNo. 14 (REVISED)\n\nCONTENT\n\nParagraph\n\nIntroduction\n\nCharge of property tax\n\nMeanings of “Owners”\nMeanings of “Land 

##### using unstructured.io

In [32]:
from unstructured.partition.pdf import partition_pdf
from unstructured.staging.base import elements_to_md

# Convert PDF to Markdown
directory_path = "./output_files/ird_pdfs"
output_directory = "./output_files/ird_md_unstructuredio"
for filename in os.listdir(directory_path):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(directory_path, filename)
        md_filename = filename.replace(".pdf", ".md")
        md_path = os.path.join(output_directory, md_filename)
        
        elements = partition_pdf(filename=pdf_path, stragtegy="hi_res")
        elements_to_md(elements=elements, filename=md_path)


### TokenTextSplitter research

In [5]:
import sys
sys.path.append('/Users/tongcc/dev/projects/interview/ird_data_pipeline')
from llama_index.core.node_parser import TokenTextSplitter
from src.config.settings import CHUNK_SIZE, CHUNK_OVERLAP

In [6]:
splitter = TokenTextSplitter(
    chunk_size=CHUNK_SIZE, 
    chunk_overlap=CHUNK_OVERLAP,
)
token_nodes = splitter.get_nodes_from_documents(documents=documents, show_progress=True)

/Users/tongcc/miniconda3/envs/timothy_koh_rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Parsing nodes: 100%|██████████| 1142/1142 [00:00<00:00, 2015.35it/s]


In [7]:
token_nodes[:5]

[TextNode(id_='e1b48477-8502-418e-a932-c3fbd03127b0', embedding=None, metadata={'file_path': '/Users/tongcc/dev/projects/interview/ird_data_pipeline/tests/output_files/ird_pdfs/dipn01.pdf', 'file_name': 'dipn01.pdf', 'file_type': 'application/pdf', 'file_size': 502041, 'creation_date': '2020-09-30', 'last_modified_date': '2020-09-30'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='57e57e01-b5b6-4b9e-8e37-8a97d6b68470', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/Users/tongcc/dev/projects/interview/ird_data_pipeline/tests/output_files/ird_pdfs/dipn01.pdf', 'file_name': 'dipn01.pdf', 'file_type': 'application/pdf', 'file_size': 502041, 'creation_date': '2020-09-30', 'last_modified_date': 

### Generating embedding research with Llama-Index

In [8]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import torch
import sys
sys.path.append('/Users/tongcc/dev/projects/interview/ird_data_pipeline')
from src.config.settings import EMBEDDING_MODEL_NAME, MAX_LENGTH

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [10]:

embedding_model = HuggingFaceEmbedding(model_name=EMBEDDING_MODEL_NAME, max_length=MAX_LENGTH, device=device)

In [11]:
embeddings = embedding_model.get_text_embedding("box")
dim = len(embeddings)
print("embedding dimension of example text ===>",dim)

embedding dimension of example text ===> 768


### Opensearch research with Llama-Index

In [12]:
from llama_index.vector_stores.opensearch import (
    OpensearchVectorStore,
    OpensearchVectorClient,
)

In [22]:
opensearch_endpoint = "http://localhost:9200"
index_name = "my-nlp-index"

text_field = "context_text" # OpensearchVectorClient store text in this field by default
embedding_field = "passage_embedding" # OpensearchVectorClient store embedding vector in this field by default
search_pipeline="nlp-search-pipeline"

opensearch_client = OpensearchVectorClient(
    endpoint=opensearch_endpoint,
    index=index_name,
    dim=dim,
    embedding_field=embedding_field,
    text_field=text_field,
    search_pipeline=search_pipeline,
)

In [23]:
# initialize the vector store
vector_store = OpensearchVectorStore(opensearch_client)

### VectorStoreIndex and Store Embeddings with Llama-Index

In [24]:
from llama_index.core import VectorStoreIndex, StorageContext

In [25]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [26]:
index = VectorStoreIndex(
    token_nodes, storage_context=storage_context, embed_model=embedding_model
)

### Retriever with Llama-Index

In [27]:
from llama_index.core.vector_stores.types import VectorStoreQueryMode
# import sys
# sys.path.append('/Users/tongcc/dev/projects/interview/ird_data_pipeline')
# from src.config.settings import TOP_K
TOP_K = 3

In [28]:
retriever = index.as_retriever(
    similarity_top_k=TOP_K,
    vector_store_query_mode=VectorStoreQueryMode.HYBRID
)

In [29]:
# Retrieve relevant documents
query = "What is the tax treatment of a gain arising from the sale of a capital asset?"

prompt = retriever.retrieve(query)

for r in prompt:
    print(r.metadata)
    print(r)
    print()

{'file_path': '/Users/tongcc/dev/projects/interview/ird_data_pipeline/tests/output_files/ird_pdfs/dipn42.pdf', 'file_name': 'dipn42.pdf', 'file_type': 'application/pdf', 'file_size': 458508, 'creation_date': '2021-06-08', 'last_modified_date': '2021-06-08'}
Node ID: 20e9ce9a-1504-460f-8519-50b2dcf7e268
Text: 85. Exchange gains or losses are neither taxable nor allowable
if they are of a capital nature. In *CIR v General Garment Manufactory
(Hong Kong) Ltd 4 HKTC 532, the exchange loss was found deductible
because, notwithstanding the Board’s limited analysis, the intention
at the time of acquisition of the foreign currency was to dispose of
it quick...
Score:  1.000


{'file_path': '/Users/tongcc/dev/projects/interview/ird_data_pipeline/tests/output_files/ird_pdfs/dipn42.pdf', 'file_name': 'dipn42.pdf', 'file_type': 'application/pdf', 'file_size': 458508, 'creation_date': '2021-06-08', 'last_modified_date': '2021-06-08'}
Node ID: c57e3f1d-2903-4cfd-9951-5af2c554692c
Text: and the gain 